**1.1. Install Essentials**

In [ ]:
from IPython.display import clear_output    #Because of the Error : NameError: name 'clear_output' is not defined

In [ ]:
!pip install tensorflow_addons
!pip install keras
!pip install segmentation-models
!pip install tf_explain
clear_output()

**1.2. Import Essentials**

In [ ]:
#Because of the Error AttributeError: module 'keras.utils' has no attribute 'generic_utils'

!pip install -U -q segmentation-models
!pip install -q tensorflow==2.2.1
!pip install -q keras==2.5
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
from tensorflow import keras
import segmentation_models as sm

In [ ]:
!pip install nibabel
!pip install segmentation-models
import tensorflow
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda, Activation
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
import numpy as np
import nibabel as nib
import glob
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from datetime import datetime
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.losses import bce_dice_loss
from segmentation_models.losses import binary_focal_dice_loss
from keras.losses import binary_crossentropy
#from keras.losses import dice_ce
import keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from res_unet_model import multi_unet_model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import random
import glob
from matplotlib import pyplot as plt
import keras
import numpy as np
import os
import math
import pathlib

**1.3. Drive Mount Dataset : Nifti images + Nifti masks**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from skimage.transform import resize
import numpy as np

In [ ]:
def load_image(image, SIZE):
    return np.round(resize(img_to_array(load_img(image))/255.,(SIZE, SIZE)),4)

In [ ]:
def load_images(image_paths, SIZE, mask=False, trim=None):
    if trim is not None:
        image_paths = image_paths[:trim]

    if mask:
        images = np.zeros(shape=(len(image_paths), SIZE, SIZE, 1))
    else:
        images = np.zeros(shape=(len(image_paths), SIZE, SIZE, 3))

    for i,image in enumerate(image_paths):
        img = load_image(image,SIZE)
        if mask:
            images[i] = img[:,:,:1]
        else:
            images[i] = img

    return images

In [ ]:
def show_mask_only(mask, cmap=None, alpha=0.4):
    plt.imshow(tf.squeeze(mask), cmap=cmap, alpha=alpha)
    plt.axis('off')

**1.4. Import NifTi Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_dir = '/content/drive/MyDrive/HarP'
path_dir = pathlib.Path('/content/drive/MyDrive/imageandsegment/')

class_names = np.array(sorted([item.name for item in path_dir.glob('*')]))
print(class_names)
IMAGE_SIZE= (256,256)
import glob   #Because of the Error : function has no Attribute glob

image_paths = sorted(glob.glob(('/content/drive/MyDrive/imageandsegment/training/image/*')))
mask_paths = sorted(glob.glob(('/content/drive/MyDrive/imageandsegment/training/mask/*')))

In [ ]:
#Approximate Running time = 4min

In [ ]:
SIZE = 256

In [ ]:
images = load_images(image_paths, SIZE)
masks = load_images(mask_paths, SIZE, mask=True)

**2.1. Defining Attention UNet for Semantic Segmentation ----> Encoder Part**

In [ ]:
from tensorflow.keras.layers import Layer   #Because of the error : NameError: name 'Layer' is not defined

In [ ]:
from keras.callbacks import Callback    #Because Callback was not defined

from keras.layers import Add    #Because of the NameError : name 'Add' is not defined

from keras.layers import Multiply #Same for Multiply

from tensorflow.keras.layers import Conv2D, MaxPool2D
#Because of the Error : NameError: name 'MaxPool2D' is not defined

**Interestingly, this module indicates and also compares each epoch's results**

In [ ]:
class ShowProgress(Callback):
    def on_epoch_end(self, epochs, logs=None):
        id = np.random.randint(200)
        exp = GradCAM()
        image = images[id]
        mask = masks[id]
        pred_mask = self.model.predict(image[np.newaxis,...])
        cam = exp.explain(
            validation_data=(image[np.newaxis,...], mask),
            class_index=1,
            model=self.model
        )
        plt.figure(figsize=(10,5))

        plt.subplot(1,2,1)
        plt.title("Original Mask")
        show_mask_only(mask, cmap='gray')

        plt.subplot(1,2,2)
        plt.title("Predicted Mask")
        show_mask_only(pred_mask, cmap='gray')

        plt.tight_layout()
        plt.show()

**2.2. Define Residual Recurrent Module**

In [ ]:
def Residual_block(x, filters, conv_layers=2):
    for i in range(conv_layers):
        if i == 0:
            residual = x
            x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
        else:
            residual = x
            x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
            x = BatchNormalization()(x)
            x = Add()([residual, x])
            x = Activation('relu')(x)
    return x

def Recurrent_block(x, filters, conv_layers=1):  # Update conv_layers to 1 for one recurrent block
    for i in range(conv_layers):
        if i == 0:
            x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
        else:
            residual = x
            x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
            x = BatchNormalization()(x)
            x = Add()([residual, x])
            x = Activation('relu')(x)
    return x

def RRCNN_block(x, filters, conv_layers=2):
    x1 = Conv2D(filters, kernel_size=(1, 1), strides=1, padding='same')(x)
    x2 = Recurrent_block(x1, filters, conv_layers)  # Use Recurrent_block for the first block
    x3 = Residual_block(x2, filters, conv_layers)  # Use Residual_block for the second block
    out = Add()([x1, x3])
    return out

**2.3. Define Squeeze-and-Excitation Mechanism and merge SE+RECURRENT+RESIDUAL**

**2.4. Define the final Architecture of the proposed Algorithm**

In [ ]:
import os
import tensorflow as tf
from datetime import datetime
import keras
from keras import backend, optimizers
from tensorflow.keras.utils import normalize
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
from keras.layers import *
from keras.models import *


# spatial squeeze by mean and channel excitation


def cse_block(prevlayer, prefix):
    mean = Lambda(lambda xin: K.mean(xin, axis=[1, 2]))(prevlayer)  # H W
    # K.int_shape() Returns the shape of tensor or variable as a tuple of int or None entries
    lin1 = Dense(K.int_shape(prevlayer)[
                 3] // 2, name=prefix + 'cse_lin1', activation='relu')(mean)
    lin2 = Dense(K.int_shape(prevlayer)[
                 3], name=prefix + 'cse_lin2', activation='sigmoid')(lin1)
    x = Multiply()([prevlayer, lin2])
    return x

# channel squeeze and spatial excitation


def sse_block(prevlayer, prefix):
    # Bug? Should be 1 here?
    conv = Conv2D(K.int_shape(prevlayer)[3], (1, 1), padding="same", kernel_initializer="he_normal",
                  activation='sigmoid', strides=(1, 1),
                  name=prefix + "_conv")(prevlayer)
    conv = Multiply(name=prefix + "_mul")([prevlayer, conv])
    return conv

# concurrent spatial and channel squeeze and channel excitation

def csse_block(x, prefix):
    cse = cse_block(x, prefix)
    sse = sse_block(x, prefix)
    x = Add(name=prefix + "_csse_mul")([cse, sse])

    return x



def Residual_block(x, filters, conv_layers=2):
    for i in range(conv_layers):
        if i == 0:
            residual = x
            x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
        else:
            residual = x
            x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
            x = BatchNormalization()(x)
            x = Add()([residual, x])
            x = Activation('relu')(x)
    return x

def Recurrent_block(x, filters, conv_layers=1):  # Update conv_layers to 1 for one recurrent block
    for i in range(conv_layers):
        if i == 0:
            x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
        else:
            residual = x
            x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
            x = BatchNormalization()(x)
            x = Add()([residual, x])
            x = Activation('relu')(x)
    return x

def RRCNN_block(x, filters, conv_layers=2):
    x1 = Conv2D(filters, kernel_size=(1, 1), strides=1, padding='same')(x)
    x2 = Recurrent_block(x1, filters, conv_layers)  # Use Recurrent_block for the first block
    x3 = Residual_block(x2, filters, conv_layers)  # Use Residual_block for the second block
    out = Add()([x1, x3])
    return out




# Feature Pyramid Model
def Feature_Pyr_R2_unet(input_shape = (256,256,3), nClasses=1, dropout_rate=0.0, batch_norm=True):
    # encoder

    #inputs = Input(shape=(width, height, input_channels))
    conv_layers=2
    filters=14
    UP_SAMP_SIZE =2
    inputs = layers.Input(input_shape, dtype=tf.float32)
    conv1 = RRCNN_block(inputs, filters, conv_layers=conv_layers) # (224,224,14)
    csse1 = csse_block(conv1, prefix="csse1")
    pool1 = MaxPooling2D((2, 2))(csse1) # (112,112,14)
    #pool1 = AveragePooling2D((2, 2))(conv1)
    conv2 = RRCNN_block(pool1, filters * 2, conv_layers=conv_layers) # (112,112,28)
    csse2 = csse_block(conv2, prefix="csse2")
    pool2 = MaxPooling2D((2, 2))(csse2) # (56,56,28)
    #pool2 = AveragePooling2D((2, 2))(conv2)

    conv3 = RRCNN_block(pool2, filters * 4, conv_layers=conv_layers) # (56,56,56)
    csse3 = csse_block(conv3, prefix="csse3")
    pool3 = MaxPooling2D((2, 2))(csse3) # (28,28,56)
    #pool3 = AveragePooling2D((2, 2))(conv3)

    conv4 = RRCNN_block(pool3, filters * 8, conv_layers=conv_layers)  # (28,28,112)
    csse4 = csse_block(conv4, prefix="csse4")
    pool4 = MaxPooling2D((2, 2))(csse4)  # (14,14,112)
    #pool4 = AveragePooling2D((2, 2))(conv4)

    conv5 = RRCNN_block(pool4, filters * 16, conv_layers=conv_layers) # (14,14,224)
    csse5 = csse_block(conv5, prefix="csse5")
    conv6=  RRCNN_block(csse5, filters * 16, conv_layers=conv_layers) # (14,14,224)
    csse6 = csse_block(conv6, prefix="csse6")

    conv7=  RRCNN_block(csse6, filters * 8, conv_layers=conv_layers)  # (14,14,112)
    csse7 = csse_block(conv7, prefix="csse7")
    net_up1 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last",interpolation='bilinear')(csse7)  # (28,28,112)
    new_concat1 = layers.concatenate([net_up1,csse4], axis=3)	 # (28,28,112)
    conv8=  RRCNN_block(new_concat1, filters * 8, conv_layers=conv_layers)  # (28,28,112)
    conv9=  RRCNN_block(conv8, filters * 8, conv_layers=conv_layers)   # (28,28,112)

    conv10 = RRCNN_block(conv9, filters * 4, conv_layers=conv_layers)   # (28,28,56)
    net_up2 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last",interpolation='bilinear')(conv10) # (56,56,56)
    new_concat2 = layers.concatenate([net_up2,csse3], axis=3)
    conv11 = RRCNN_block(new_concat2, filters * 4, conv_layers=conv_layers)  #(56,56,56)
    conv12 = RRCNN_block(conv11, filters * 4, conv_layers=conv_layers)   #(56,56,56)


    conv13 = RRCNN_block(conv12, filters * 2, conv_layers=conv_layers)   # (56,56,28)
    net_up3 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last",interpolation='bilinear')(conv13) # (112,112,28)
    new_concat3 = layers.concatenate([net_up3,csse2], axis=3)
    conv14 = RRCNN_block(new_concat3, filters * 2, conv_layers=conv_layers)  #(112,112,28)
    conv15 = RRCNN_block(conv14, filters * 2, conv_layers=conv_layers)   #(112,112,28)

    conv16 = RRCNN_block(conv15, filters , conv_layers=conv_layers)   # (112,112,14)

    # Feature Stacking
    pyr_layer1 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last",interpolation='bilinear')(conv16)
    pyr_layer2= layers.UpSampling2D(size=(UP_SAMP_SIZE*2, UP_SAMP_SIZE*2), data_format="channels_last",interpolation='bilinear')(conv13)
    pyr_layer3= layers.UpSampling2D(size=(UP_SAMP_SIZE*4, UP_SAMP_SIZE*4), data_format="channels_last",interpolation='bilinear')(conv10)
    pyr_layer4= layers.UpSampling2D(size=(UP_SAMP_SIZE*8, UP_SAMP_SIZE*8), data_format="channels_last",interpolation='bilinear')(csse7)



    concat1 = layers.concatenate([pyr_layer1,pyr_layer2,pyr_layer3,pyr_layer4], axis=3)
    conv17 = RRCNN_block(concat1, filters*4, conv_layers=conv_layers)  #(224,224,56)
    conv18 = RRCNN_block(conv17, filters, conv_layers=conv_layers)   #(224,224,14)

    conv_final = layers.Conv2D(nClasses, kernel_size=(1,1))(conv18)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)

    # Model integration
    model = models.Model(inputs, conv_final, name="Feature_Pyr_R2_unet")
    return model

def final_model(input_shape):
    final_unet_model = Feature_Pyr_R2_unet(input_shape=(input_shape[0], input_shape[1], 3))
    return final_unet_model

from tensorflow.keras.utils import plot_model


model = final_model(input_shape = (256,256,3))
model.summary()

# Callbacks
cb = [
    # EarlyStopping(patience=3, restore_best_weight=True), # With Segmentation I trust on eyes rather than on metrics
    ModelCheckpoint("AttentionCustomUNet.keras", save_best_only=True),
    ShowProgress()
]

**2.5. Define Performance Metrics and Compile the model**

In [ ]:
# Metrics
from keras.metrics import MeanIoU     #Because of the Error NameError: name 'MeanIoU' is not defined

In [ ]:
import tensorflow as tf

def dice_coeff(y_true, y_pred):
    smooth = 1e-15
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer='adam',
    metrics=['accuracy', dice_coeff, tf.keras.metrics.MeanIoU(num_classes=2, name='IoU')]
)

In [ ]:
from tf_explain.core.grad_cam import GradCAM
#Because of the Error NameError: name 'GradCAM' is not defined

2.6. Start the Process of training

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)


# Config Training
BATCH_SIZE = 32
SPE = len(X_train) // BATCH_SIZE

# Training
results = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=61, # 15 will be enough for a good Model for better model go with 20+
    steps_per_epoch=SPE,
    batch_size=BATCH_SIZE,
    callbacks=cb
)

In [ ]:
loss, accuracy, iou, val_loss, val_accuracy, val_iou = results.history.values()

In [ ]:
import cv2
import numpy as np

def get_mask_box(mask):
    img = np.uint8(mask[0, :, :, 0] * 255)

    _, binary = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        color = (0, 255, 0)
        cv2.rectangle(img_bgr, (x, y), (x + w, y + h), color, 2)

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        thickness = 1
        text_height = f'Height: {h}'
        text_width = f'Width: {w}'
        text_height_size = cv2.getTextSize(text_height, font, font_scale, thickness)[0]
        text_width_size = cv2.getTextSize(text_width, font, font_scale, thickness)[0]
        cv2.putText(img_bgr, text_height, (x, y - text_height_size[1]), font, font_scale, color, thickness, cv2.LINE_AA)
        cv2.putText(img_bgr, text_width, (x + w - text_width_size[0], y + h + text_width_size[1]), font, font_scale, color, thickness, cv2.LINE_AA)

    plt.imshow(img_bgr,cmap = None)
    plt.axis('off')

# **3. Results**

**3.1. Show the predicted masks and compare them to the original ones**

In [ ]:
plt.figure(figsize=(20,25))
n=0
for i in range(1,(5*3)+1):
    plt.subplot(5,3,i)
    if n==0:
        id = np.random.randint(len(images))
        image = images[id]
        mask = masks[id]
        pred_mask = model.predict(image[np.newaxis,...])

        plt.title("Original Mask")
        show_mask(image, mask)
        n+=1
    elif n==1:
        plt.title("Predicted Mask")
        show_mask(image, pred_mask)
        n+=1
    elif n==2:
        pred_mask = (pred_mask>0.5).astype('float')
        plt.title("Processed Mask")
        show_mask(image, pred_mask)
        n=0
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,25))
n=0
for i in range(1,(5*3)+1):
    plt.subplot(5,3,i)
    if n==0:
        id = np.random.randint(len(images))
        image = images[id]
        mask = masks[id]
        pred_mask = model.predict(image[np.newaxis,...])

        plt.title("Original Mask")
        show_mask(image, mask)
        n+=1
    elif n==1:
        plt.title("Predicted Mask")
        show_mask(image, pred_mask)
        n+=1
    elif n==2:
        pred_mask = (pred_mask>0.5).astype('float')
        plt.title("Processed Mask")
#         show_mask(image, pred_mask)
        get_mask_box(pred_mask)
        n=0
plt.tight_layout()
plt.show()

**3.2. Just a Sample (if you have anything especific in mind)**

In [ ]:
id = 12
image = images[id]
mask = model.predict(image[np.newaxis,...])
get_mask_box(mask)

**3.3. And the RESULTS...**

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# assume y_true is the true binary mask and y_pred is the predicted mask
# both have shape (num_samples, height, width, num_channels)
y_true = y_val
y_pred = model.predict(X_val)

# threshold the predicted mask to convert to binary values
y_true_bool = (y_true > 0.5).astype(bool)
cm = confusion_matrix(y_true_bool.flatten(), y_pred_thresh.flatten())
cr = classification_report(y_true_bool.flatten(), y_pred_thresh.flatten())

print("Confusion matrix:")
print(cm)
print("Classification report:")
print(cr)

In [ ]:
#Plotting Model Loss (Training Loss and Validation Loss)

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('2D U-Net Model Loss')
plt.ylabel('Loss')
plt.xlabel('Number of Epochs')
plt.legend(['Training Loss' , 'Validation Loss'], loc = 'upper right')
plt.show()

In [ ]:
#Plotting Model Accuracy (Training Accuracy and Validation Accuracy)

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('2D U-Net Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Number of Epochs')
plt.legend(['Training Loss' , 'Validation Loss'], loc = 'lower right')
plt.show()

In [ ]:
#Plotting Model Dice Similarity Coefficient

plt.plot(hist.history['dice_coef1'])
plt.plot(hist.history['val_dice_coef1'])
plt.title('2D Model Dice Similarity Coefficient')
plt.ylabel('Dice Similarity Coefficient')
plt.xlabel('Number of Epochs')
plt.legend(['Trainining DSC' , 'Validation DSC'], loc = 'lower right')
plt.show()

In [ ]:
#Plotting Intersection over Union

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('2D U-Net Model IoU')
plt.ylabel('Intersection over Union')
plt.xlabel('Number of Epochs')
plt.legend(['Training IoU' , 'Validation IoU'], loc = 'upper right')
plt.show()